In [1]:
import sympy as sp
import numpy as np
from sympy.physics.vector import dynamicsymbols
from sympy.physics.vector import time_derivative
from sympy.physics.vector import ReferenceFrame
N = ReferenceFrame('N')
import pylab as pl
import control
from EoM import *
from sympy.physics.mechanics import *
from numpy.linalg import matrix_rank, eig
import math
import intelligent_robotics as ir

In [2]:
def rot_x(theta):
    rot_x = sp.Matrix([[1,0,0],
                       [0,sp.cos(theta),-sp.sin(theta)],
                       [0,sp.sin(theta),sp.cos(theta)]])
    return rot_x

def rot_y(theta):
    rot_y = sp.Matrix([[sp.cos(theta),0,sp.sin(theta)],
                       [0,1,0],
                       [-sp.sin(theta),0,sp.cos(theta)]])
    return rot_y

def rot_z(theta):
    rot_z = sp.Matrix([[sp.cos(theta),-sp.sin(theta),0],
                       [sp.sin(theta),sp.cos(theta),0],
                       [0,0,1]])
    return rot_z

def trans(a,b,c):
    x = sp.Matrix([[a],[b],[c]])
    return x

def get_torque_from_L(L,q,qd):
    round_L_round_q = sp.zeros(len(q),1);
    i = 0;
    for q_i in q:
        round_L_round_q_i = [];
        round_L_round_q_i = sp.diff(L,q_i);
        round_L_round_q[i] = round_L_round_q_i;
        i+=1;
     
    d_dt_round_L_round_qd = sp.zeros(len(qd),1);
    i = 0;
    for qd_i in qd:
        round_L_round_qd_i = [];
        d_dt_round_L_round_qd_i = [];
        round_L_round_qd_i = sp.diff(L,qd_i);
        d_dt_round_L_round_qd_i = time_derivative(round_L_round_qd_i,N);
        d_dt_round_L_round_qd[i] = d_dt_round_L_round_qd_i;
        i+=1;
        
    tau = d_dt_round_L_round_qd - round_L_round_q 
    return tau

In [43]:
r, l, W = sp.symbols('r, l, W')
m_w, m_b, g = sp.symbols('m_w, m_b, g')
I_w, I_by, I_bz = sp.symbols('I_w, I_by, I_bz')
theta_P, theta_Y, theta_w, theta_wr, theta_wl = dynamicsymbols('theta_P, theta_Y, theta_w, theta_wr, theta_wl')

# Displacement Vector
x_c = rot_z(theta_Y) @trans(r*theta_w,0,0)
x_w = x_c +  trans(0,0,r)
x_wl = x_w +  rot_z(theta_Y) @trans(0,W/2,0)
x_wr = x_w +  rot_z(theta_Y) @trans(0,-W/2,0)
x_b = x_w +  rot_z(theta_Y) @rot_y(theta_P) @ trans(0,0,l)



# Velocity Vector
v_wl= sp.simplify(time_derivative(x_wl,N))
v_wr = sp.simplify(time_derivative(x_wr,N))
v_b = sp.simplify(time_derivative(x_b,N))

# theta_w = (theta_wl + theta_wr)/2

# Angular Velocity Vector
w_w = trans(0, theta_w.diff(), 0)
w_wr = trans(0, theta_wr.diff(), 0)
w_wl = trans(0, theta_wl.diff(), 0)
w_b = trans(0, theta_P.diff(), theta_Y.diff())

# Generalized Coordinates
q = sp.Matrix([[theta_w], [theta_P], [theta_Y]])
qd = q.diff()
qdd = qd.diff()

# Kinetic Energy
T_trans = 0.5*(m_w*v_wl.dot(v_wl) + m_w*v_wr.dot(v_wr) + m_b*v_b.dot(v_b))
#T_body = I_by * w_b[1]**2 
#T_wheel = I_wy * w_w[1]**2

T_body = I_by * w_b[1]**2 + I_bz * w_b[2]**2 
T_wheel = I_w * w_wr[1]**2 + I_w * w_wl[1]**2
#T_motor = I_my * n**2 * (theta_w.diff() - theta_P.diff())**2
#T_gimbal_right = I_lfx * w_lf[0]**2 + I_lfy * w_lf[1]**2 + I_lfz * theta_gb.diff()**2
#T_gimbal_left = I_rfx * w_rf[0]**2 + I_rfy * w_rf[1]**2 + I_rfz * theta_gb.diff()**2
T_rot = 0.5*(T_body + T_wheel)

#T_rot = 0.5*(I_wy*w_w[1]**2 + I_by*w_b[1]**2 + I_my*n*n*(theta_w.diff()**2) + I_lfz*w_lf[2]**2 + I_rfz*w_rf[2]**2 + I_lfy*w_lf[1]**2 + I_rfy*w_rf[1]**2)
#I_my*n*n*(theta_w.diff()-theta_P.diff())**2,  I_bx*(theta_R.diff()-theta_gb.diff())**2
# Potential Energy
V =  m_b*g*x_b[2]+m_w*g*x_wl[2]++m_w*g*x_wr[2]

# Lagrangian
L = T_trans + T_rot - V


In [44]:
tau = get_torque_from_L(L,q,qd)
# tau = sp.simplify(tau)
# Ml, Cl, Gl, Wl = ir.get_EoM_from_T(tau,qdd,g)
# Ml, Cl, Gl, Wl

In [55]:
sp.simplify(tau)

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                      r*(-1.0*l*m_b*sin(theta_P(t))*Derivative(theta_P(t), t)**2 - 1.0*l*m_b*sin(theta_P(t))*Derivative(theta_Y(t), t)**2 + 1.0*l*m_b*cos(theta_P(t))*Derivative(theta_P(t), (t, 2)) - 1.0*m_b*r*theta_w(t)*Derivative(theta_Y(t), t)**2 + 1.0*m_b*r*Derivative(theta_w(t), (t, 2)) - 2.0*m_w*r*theta_w(t)*Derivative(theta_Y(t), t)**2 + 2.0*m_w*r*Derivative(theta_w(t), (t, 2)))],
[                                                                                                                                                                                       

In [46]:
T_r, T_l = sp.symbols('T_r, T_l')
u = sp.Matrix([[T_r], [T_l]])

u_matrix = sp.Matrix([[-T_r-T_l], [T_r+ T_l], [D * (T_r - T_l)/r]])

In [54]:
tau_eq = tau+u_matrix
sp.simplify(tau_eq)

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                 -1.0*T_l - 1.0*T_r - 1.0*l*m_b*r*sin(theta_P(t))*Derivative(theta_P(t), t)**2 - 1.0*l*m_b*r*sin(theta_P(t))*Derivative(theta_Y(t), t)**2 + 1.0*l*m_b*r*cos(theta_P(t))*Derivative(theta_P(t), (t, 2)) - 1.0*m_b*r**2*theta_w(t)*Derivative(theta_Y(t), t)**2 + 1.0*m_b*r**2*Derivative(theta_w(t), (t, 2)) - 2.0*m_w*r**2*theta_w(t)*Derivative(theta_Y(t), t)**2 + 2.0*m_w*r**2*Derivative(theta_w(t), (t, 2))],
[                                                                                                                                                          

In [48]:
M, C, G, W = get_EoM_from_T(tau_eq,qdd,g, u)

In [49]:
M

Matrix([
[       r**2*(m_b + 2.0*m_w), 1.0*l*m_b*r*cos(theta_P(t)),                                                                                            0],
[1.0*l*m_b*r*cos(theta_P(t)),         1.0*I_by + l**2*m_b,                                                                                            0],
[                          0,                           0, 1.0*I_bz + m_b*(l*sin(theta_P(t)) + r*theta_w(t))**2 + 0.5*m_w*(W**2 + 4*r**2*theta_w(t)**2)]])

In [50]:
C

Matrix([
[                                                                                           -r*(1.0*l*m_b*sin(theta_P(t))*Derivative(theta_P(t), t)**2 + 1.0*l*m_b*sin(theta_P(t))*Derivative(theta_Y(t), t)**2 + 1.0*m_b*r*theta_w(t)*Derivative(theta_Y(t), t)**2 + 2.0*m_w*r*theta_w(t)*Derivative(theta_Y(t), t)**2)],
[                                                                                                                                                                                                                             -1.0*l*m_b*(l*sin(theta_P(t)) + r*theta_w(t))*cos(theta_P(t))*Derivative(theta_Y(t), t)**2],
[(1.0*l**2*m_b*sin(2*theta_P(t))*Derivative(theta_P(t), t) + 2.0*l*m_b*r*theta_w(t)*cos(theta_P(t))*Derivative(theta_P(t), t) + 2.0*l*m_b*r*sin(theta_P(t))*Derivative(theta_w(t), t) + 2.0*m_b*r**2*theta_w(t)*Derivative(theta_w(t), t) + 4.0*m_w*r**2*theta_w(t)*Derivative(theta_w(t), t))*Derivative(theta_Y(t), t)]])

In [15]:
G

Matrix([
[                       0],
[-g*l*m_b*sin(theta_P(t))],
[                       0]])

In [16]:
W

Matrix([
[   1,   1],
[  -1,  -1],
[-D/r, D/r]])

In [51]:
linearlize_eq = {sp.sin(theta_P):theta_P, sp.cos(theta_P):1, theta_P.diff():0, sp.sin(theta_Y):theta_Y, sp.cos(theta_Y):1, theta_Y.diff():0}
Ml = M.subs(linearlize_eq)
Cl = C.subs(linearlize_eq)
Gl = G.subs(linearlize_eq)
Wl = W.subs(linearlize_eq)

Ml

Matrix([
[r**2*(m_b + 2.0*m_w),         1.0*l*m_b*r,                                                                                       0],
[         1.0*l*m_b*r, 1.0*I_by + l**2*m_b,                                                                                       0],
[                   0,                   0, 1.0*I_bz + m_b*(l*theta_P(t) + r*theta_w(t))**2 + 0.5*m_w*(W**2 + 4*r**2*theta_w(t)**2)]])

In [52]:
Cl

Matrix([
[0],
[0],
[0]])

In [53]:
Gl

Matrix([
[                  0],
[-g*l*m_b*theta_P(t)],
[                  0]])

In [18]:
param = {I_w:0.199893505, I_by:2.139168508, I_bz:2.139168508, m_b:40, m_w:15, r:0.165,l:0.3,  g:9.81,  D:0.165}

Mlp = msubs(Ml, param)
Clp = msubs(Cl, param)
Glp = msubs(Gl, param)
Wlp = msubs(Wl, param)

Mlp_inv = Mlp.inv()
qdd_rhs_A = Mlp_inv*(-Clp -Glp)
qdd_rhs_B = Mlp_inv*Wlp*u

X = q.col_join(qd)
Xd_A = qd.col_join(qdd_rhs_A)
Xd_B = qd.col_join(qdd_rhs_B)
U = u

A = Xd_A.jacobian(X)
B = Xd_B.jacobian(U)
C = X.jacobian(X)
D = X.jacobian(U)
B
# ss0 = [A, B, C, D]
# sys0 = control.ss(*[pl.array(mat_i.subs(param)).astype(float) for mat_i in ss0])
# mprint(sys0)

Matrix([
[                                                                                          0,                                                                                          0],
[                                                                                          0,                                                                                          0],
[                                                                                          0,                                                                                          0],
[                                                                          0.945484916261624,                                                                          0.945484916261624],
[                                                                         -0.500431400505938,                                                                         -0.500431400505938],
[-1.0/(3.6*theta_P(t)**2 + 3.96*theta_P(t)*theta_w(t) + 

In [19]:
Q = sp.Matrix([ [0.3,  0,    0,    0,    0,    0],
                [0,    0.3,  0,    0,    0,    0],
                [0,    0,    0.01, 0,    0,    0],
                [0,    0,    0,    0.1,  0,    0],
                [0,    0,    0,    0,    0.1,  0],
                [0,    0,    0,    0,    0,    0.01]])

R = sp.Matrix([ [1, 0],
                [0, 1]])

In [111]:
K, S, E = control.lqr(A, B, Q, R)

In [113]:
K

array([[ 8.76345976e-16, -5.47722558e-01, -7.68659094e+01,
        -2.51964991e-01, -1.12736476e+00, -1.53373831e+01],
       [-5.47722558e-01, -1.87198599e-14, -7.68659094e+01,
        -1.12736476e+00, -2.51964991e-01, -1.53373831e+01]])

In [116]:
K[0]

array([ 8.76345976e-16, -5.47722558e-01, -7.68659094e+01, -2.51964991e-01,
       -1.12736476e+00, -1.53373831e+01])

In [118]:
K[0][2:6]

array([-76.86590936,  -0.25196499,  -1.12736476, -15.33738306])